In [1]:
const dfd = require("danfojs-node");
const tf = dfd.tensorflow
// const tf = require('@tensorflow/tfjs');

In [2]:
config = {
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  // maxBatchSize: 13,
  maxBatchSize: 1,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 50,
  timeCol: 'tanggal',
  feature: ['rr']
}

// usecols = [config.timeCol].concat(config.feature)

function trainTestSplit(dataset, timeStep = 1) {
  dataX = []
  dataY = []

  // for (let i = 0; i < dataset.values.length - timeStep - 1; i++) {
  for (let i = 0; i < dataset.values.length - timeStep; i++) {
    dataX.push(dataset.iloc({ rows: [`${i}:${i + timeStep}`] }).values)
    dataY.push(dataset.iloc({ rows: [i + timeStep] }).values[0])
  }

  return [dataX, dataY]
}

{
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 1,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 50,
  timeCol: 'tanggal',
  feature: [ 'rr' ]
}

In [3]:
$$async$$ = true;
(async function () {
  dataset = await dfd.readCSV("../Data/1985-2021.csv")
  dataset = dataset.iloc({rows:[`${config.rowStart}:${config.rowEnd}`]})
  datelist = dataset.loc({columns: [config.timeCol]})

  $$done$$(dataset.shape)
})()

[ 60, 11 ]

In [4]:
featureset = dataset.loc({columns: config.feature})

scaller = new dfd.MinMaxScaler()
scaller.fit(featureset)

featuresetScaled = scaller.transform(featureset)

trainSize = featuresetScaled.values.length * 0.9
trainset = featuresetScaled.iloc({rows: [`${0}:${trainSize}`]})
testset =  featuresetScaled.iloc({rows: [`${trainSize}:${featuresetScaled.values.length}`]})

trainset = trainTestSplit(trainset, timeStep=config.timesteps)
testset = trainTestSplit(testset, timeStep=config.timesteps)

XTrain = tf.tensor(trainset[0])
yTrain = tf.tensor(trainset[1])

XTest = tf.tensor(testset[0])
yTest = tf.tensor(testset[1])

XTrain = tf.reshape(XTrain, [XTrain.shape[0], config.feature.length, XTrain.shape[1]])
XTest = tf.reshape(XTest, [XTest.shape[0], config.feature.length, XTest.shape[1]])

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 1, 2 ],
  dtype: 'float32',
  size: 8,
  strides: [ 2, 2 ],
  dataId: {},
  id: 18,
  rankType: '3',
  scopeId: 6
}

In [9]:
// yPredLoss = []
// yTrueLoss = []
// loss = []

// function meanSquaredError(yTrue, yPred) {
//   yPredLoss.push(yPred.array())
//   yTrueLoss.push(yTrue.array())
//   loss.push(tf.losses.meanSquaredError(yTrue, yPred).arraySync())
//   return tf.losses.meanSquaredError(yTrue, yPred)
// }

model = tf.sequential()

model.add(tf.layers.lstm({
  units: config.unitSize,
  batchInputShape: [config.maxBatchSize, config.feature.length, config.timesteps],
  goBackwards: true,
  dropout: config.dropout,
  weights: [
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774],
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.zeros([4])
  ]
}))

// model.layers[0].getWeights()[0].print()

model.compile({
  optimizer: tf.train.sgd(config.learningRate),
  loss: 'meanSquaredError',
  // loss: meanSquaredError,
  runEagerly: true
})

model.summary()

__________________________________________________________________________________________
Layer (type)                Input Shape               Output shape              Param #   
lstm_LSTM2 (LSTM)           [[1,1,2]]                 [1,1]                     16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
__________________________________________________________________________________________


In [10]:
// batchEnd = []
epochEnd = []

$$async$$ = true;
(async function () {
  history = await model.fit(XTrain, yTrain, {
    shuffle: false,
    epochs: config.maxEpochs,
    verbose: 1,
    batchSize: config.maxBatchSize,
    callbacks: [
      new tf.CustomCallback({
        // onEpochBegin: null,
        onEpochEnd: (epoch, logs = null) => epochEnd.push(logs.loss),
        // on_batch_begin: null,
        // onBatchEnd: (batch, logs = null) => batchEnd.push(logs.loss),
        // onTrainBegin: null,
        // onTrainEnd: null,
      })
    ]
  })
  $$done$$()
})()

Epoch 1 / 50


1663ms 31974us/step - loss=0.0799 
Epoch 2 / 50


738ms 14195us/step - loss=0.0717 
Epoch 3 / 50


1009ms 19412us/step - loss=0.0690 
Epoch 4 / 50


1427ms 27444us/step - loss=0.0678 
Epoch 5 / 50


...

477ms 9181us/step - loss=0.0635 
Epoch 47 / 50


631ms 12135us/step - loss=0.0635 
Epoch 48 / 50


493ms 9487us/step - loss=0.0634 
Epoch 49 / 50


501ms 9628us/step - loss=0.0634 
Epoch 50 / 50


492ms 9461us/step - loss=0.0634 


In [11]:
epochEnd

[
  0.07985345274209976,  0.0716899037361145,  0.0690271183848381,
  0.06783075630664825,  0.0671655684709549, 0.06673196703195572,
  0.06641414016485214, 0.06616140902042389, 0.06594928354024887,
  0.06576499342918396, 0.06560125201940536, 0.06545361131429672,
  0.06531914323568344, 0.06519577652215958, 0.06508197635412216,
  0.06497654318809509, 0.06487848609685898, 0.06478700786828995,
  0.06470144540071487,  0.0646212100982666, 0.06454578787088394,
  0.06447473913431168, 0.06440768390893936, 0.06434430181980133,
  0.06428425014019012, 0.06422726064920425,  0.0641731321811676,
  0.06412162631750107, 0.06407254189252853, 0.06402570009231567,
  0.06398095935583115, 0.06393817067146301, 0.06389720737934113,
  0.06385792791843414,  0.0638202428817749, 0.06378404796123505,
   0.0637492686510086, 0.06371579319238663, 0.06368356943130493,
  0.06365251541137695, 0.06362255662679672, 0.06359364092350006,
  0.06356573104858398, 0.06353874504566193,  0.0635126605629921,
  0.06348742544651031, 

In [12]:
proyeksi = model.predict(XTest, yTest)

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 1 ],
  dtype: 'float32',
  size: 4,
  strides: [ 1 ],
  dataId: {},
  id: 1295612,
  rankType: '2',
  scopeId: 1118420
}

In [15]:
proyeksi.dataSync()

Float32Array(4) [
  0.1997341513633728,
  0.187837153673172,
  0.18746259808540344,
  0.18775293231010437
]